# **PROCESSING INCOME**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt 
import stat 
import sklearn as sk

## **1. Data Collection**

In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 
  

In [4]:
X.shape

(48842, 14)

In [7]:
y.shape

(48842, 1)

In [11]:
# meger data with columns
raw_data = pd.concat([X,y], axis= 1) 

In [12]:
raw_data.shape

(48842, 15)

In [13]:
raw_data.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


**the mean value of features**
| `Thuộc tính`         | Mô tả                                                                 |
|--------------------|----------------------------------------------------------------------|
| `age`                | Tuổi của cá nhân.                                                     |
| `workclass`         | Loại hình việc làm (ví dụ: Tư nhân, Chính phủ, Tự kinh doanh, v.v.). |
| `fnlwgt`             | Trọng số cuối cùng của khảo sát.                                     |
| `education`          | Trình độ học vấn.                                                     |
| `education-num`      | Số năm học (dưới dạng số nguyên).                                     |
| `marital-status`     | Tình trạng hôn nhân (độc thân, đã kết hôn, đã ly hôn, v.v.).          |
| `occupation`         | Nghề nghiệp (Quản lý, Hành chính, Dịch vụ, v.v.).                     |
| `relationship`       | Quan hệ với chủ hộ (vợ/chồng, con cái, người thân khác, v.v.).        |
| `race`               | Chủng tộc.                                                            |
| `sex`                | Giới tính.                                                            |
| `capital-gain`       | Thu nhập từ vốn (không phải từ lương).                               |
| `capital-loss`       | Khoản lỗ từ vốn (từ đầu tư).                                          |
| `hours-per-week`     | Số giờ làm việc mỗi tuần.                                             |
| `native-country`     | Quốc gia xuất xứ.                                                     |
"""

In [24]:
raw_data.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

In [25]:
raw_data.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income            object
dtype: object

## **2. Data Cleaning**

### **2.1 Check duplicated records**

In [ ]:
num_dup_rows = raw_data.duplicated(keep="first").sum()
num_dup_rows

In [22]:
raw_data = raw_data.drop_duplicates(keep="first")

In [23]:
raw_data.duplicated(keep="first").sum()

np.int64(0)

### **2.2 Missing value**

**how is the dataset distributed ?**

In [26]:
# missing ratio
def missing_ratio(series):
    return series.isnull().mean() * 100


def lower_quantile(series):
    return series.quantile(0.25)

def median(series):
    return series.median()


def upper_quantile(series):
    return series.quantile(0.75)

# the following function will specifically present the number distribution 
def number_dis(data):
    col = data.select_dtypes(include=['float','int'])
    return col.agg([missing_ratio,"min",lower_quantile,median,upper_quantile,"max"])
number_dis(raw_data)

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0
min,17.0,12285.0,1.0,0.0,0.0,1.0
lower_quantile,28.0,117555.0,9.0,0.0,0.0,40.0
median,37.0,178140.0,10.0,0.0,0.0,40.0
upper_quantile,48.0,237620.0,12.0,0.0,0.0,45.0
max,90.0,1490400.0,16.0,99999.0,4356.0,99.0


- there are no the missing value in number columns

In [27]:
def num_values(series):
    return series.nunique()

def value_ratios(series):
 
    # compute percent of the series without checking for missing values 
    value_counts = series.value_counts(normalize=True) * 100
    
    # convert to dictionary type and need to round the percentage
    value_ratios = value_counts.round(1).to_dict()
    
    # sort the dictionary value in ascending
    sorted_value_ratios = dict(sorted(value_ratios.items(), key=lambda item: item[1], reverse=True))
    
    return sorted_value_ratios

# the distribution in object features 
def object_dis(data):
    col = data.select_dtypes(include="object") 
    return col.agg([missing_ratio, num_values, value_ratios])

object_features_dic = object_dis(raw_data)

In [28]:
object_features_dic

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,income
missing_ratio,1.972835,0.0,0.0,1.978981,0.0,0.0,0.0,0.561326,0.0
num_values,9,16,7,15,6,5,2,42,4
value_ratios,"{'Private': 70.8, 'Self-emp-not-inc': 8.1, 'Lo...","{'HS-grad': 32.3, 'Some-college': 22.3, 'Bache...","{'Married-civ-spouse': 45.8, 'Never-married': ...","{'Prof-specialty': 12.9, 'Craft-repair': 12.8,...","{'Husband': 40.4, 'Not-in-family': 25.7, 'Own-...","{'White': 85.5, 'Black': 9.6, 'Asian-Pac-Islan...","{'Male': 66.8, 'Female': 33.2}","{'United-States': 90.3, 'Mexico': 2.0, '?': 1....","{'<=50K': 50.6, '<=50K.': 25.5, '>50K': 16.1, ..."


**Show the feature to check value**

In [34]:
object_features_dic.loc["value_ratios", "workclass"]

{'Private': 73.6,
 'Self-emp-not-inc': 8.4,
 'Local-gov': 6.8,
 'State-gov': 4.3,
 'Self-emp-inc': 3.7,
 'Federal-gov': 3.1,
 'Without-pay': 0.0,
 'Never-worked': 0.0}

I observed that there is '?' in each object attribute.
Hence, I will replace this with Nan 


In [32]:
raw_data = raw_data.replace("?", np.nan)

In [33]:
object_features_dic = object_dis(raw_data)
object_features_dic

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,income
missing_ratio,5.734128,0.0,0.0,5.754615,0.0,0.0,0.0,1.753631,0.0
num_values,8,16,7,14,6,5,2,41,4
value_ratios,"{'Private': 73.6, 'Self-emp-not-inc': 8.4, 'Lo...","{'HS-grad': 32.3, 'Some-college': 22.3, 'Bache...","{'Married-civ-spouse': 45.8, 'Never-married': ...","{'Prof-specialty': 13.4, 'Craft-repair': 13.3,...","{'Husband': 40.4, 'Not-in-family': 25.7, 'Own-...","{'White': 85.5, 'Black': 9.6, 'Asian-Pac-Islan...","{'Male': 66.8, 'Female': 33.2}","{'United-States': 91.4, 'Mexico': 2.0, 'Philip...","{'<=50K': 50.6, '<=50K.': 25.5, '>50K': 16.1, ..."


**Các bản ghi bị ảnh hưởng có nên được loại trừ khỏi tập dữ liệu không? Và nên thay thế bằng giá trị gì?**
- Analyze: 
    - there are missing values in these including: `workclass`: 5.734128 , `occupation`: 5.754615	, `native-country`: 1.753631
    - However, these values is quite small.
- Conclude: 
    - Handle it instead of removing these columns.  
    - these attribute come in object or catergorical. Therefore, I will replace them with components which appear the most  

In [36]:
missing_columns = ["workclass","occupation", "native-country"]

In [37]:
# functions which will replace by max fequency 
def replace_by_max_fequency(data, object_features_dic, features:list):
   
    for feature in features: 
        
        # lấy giá trị có tần số suất hiện cao nhất bằng cách lấy vị trị (location) của value_ratios trong hàm object_dis ở trên
        # vị trí số 0 là lớn nhất vì em đã sắp xếp các giá trị này rồi ạ.
       replace_value = list(object_features_dic.loc['value_ratios',feature])[0] 
       
       # handle Nan
       data[feature] = data[feature].replace(np.nan,replace_value )
       
    return data

raw_data = replace_by_max_fequency(raw_data,object_features_dic, missing_columns)

In [38]:
object_dis(raw_data)

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,income
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
num_values,8,16,7,14,6,5,2,41,4
value_ratios,"{'Private': 75.1, 'Self-emp-not-inc': 7.9, 'Lo...","{'HS-grad': 32.3, 'Some-college': 22.3, 'Bache...","{'Married-civ-spouse': 45.8, 'Never-married': ...","{'Prof-specialty': 18.4, 'Craft-repair': 12.5,...","{'Husband': 40.4, 'Not-in-family': 25.7, 'Own-...","{'White': 85.5, 'Black': 9.6, 'Asian-Pac-Islan...","{'Male': 66.8, 'Female': 33.2}","{'United-States': 91.5, 'Mexico': 1.9, 'Philip...","{'<=50K': 50.6, '<=50K.': 25.5, '>50K': 16.1, ..."
